# Creating a Twitter-Kafka Producer

In this notebook we are going to connect to Twitter in real-time by using the tweepy Python module. 
Afterwards a Kafka producer is created. By running the application, we listen to a certain topic on Twitter transform the incoming stream and write it to a Kafka topic.

### Preprocessing

If the import paragraph gives an error, install tweepy and kafka modules.

In [2]:
#%%bash
#pip3 install tweepy kafka

In [3]:
import tweepy
import time
from kafka import KafkaProducer
import json
import datetime

### Create Kafka producer

Within a Docker environment, you can refer to the bootstrap_servers (= Kafka server) by using kafka:29092. Doccker will resolve this to *kafka_ip*:9092.

The *topic_name* will be used in the actions block as topic name to create and send the messages to.

In [4]:
producer = KafkaProducer(bootstrap_servers='kafka:29092')
#topic_name = 'IESEG_MBD'
topic_name = 'trump'

### Create Twitter listener stream

Twitter credentials

In [5]:
api_key = "GUo8OUhS3MzukAU5WcW9J4saU"
api_secret =  "DzKro05ow6YKrtASDZph7pPCOg5wmO4Nl5uaGvgTHBfOp8zpwJ"
access_token =  "2754263935-roAaGGAJPJvggyqbP3JsxS5SiEXkhmkKR6JOj0c"
access_token_secret =  "6KLfzIUaygftQL14aTIgxoHYFKk7vE7AMw2slrcR1CcSl"

Twitter athentification based on credentials given above.

In [6]:
# Creating the authentication object
auth = tweepy.OAuthHandler(api_key, api_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object by passing in auth information
api = tweepy.API(auth) 

*Actions* - This block is crutial and defines what needs to happen every time a tweet is processed:
    - First, the twitter stream is transformed by selecting the necessary field
    - Transformation of the stream to bytes
    - Sending the stream to Kafka

In [7]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        # selection
        record = {}
        record["id"] = status.id
        record["created_at"] = datetime.datetime.strftime(status.created_at, '%Y-%m-%d %H:%M:%S.%f')
        record["text"] = status.text
        record["source"] = status.source
        record["user_id"] = status.user.id
        record["user_name"] = status.user.name
        record["user_location"] = status.user.location
        record["user_followers_count"] = status.user.followers_count
        record["user_friends_count"] = status.user.friends_count
        record["entities_hashtags"] = status.entities["hashtags"]
        record["entities_user_mentions"] = status.entities["user_mentions"]
        record["lang"] = status.lang
        # Transform to bytes
        bytes_record = json.dumps(record).encode('utf-8')
        # send to Kafka
        producer.send(topic_name, bytes_record)

### Define the stream

In [ ]:
myStreamListener = MyStreamListener
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener())

### Run stream

In [ ]:
myStream.filter(track=["trump"])